In [ ]:
import configparser
import os
import sys
import csv
import ipaddress

sys.path.insert(0, os.path.dirname(os.getcwd()))  # For Notebook

from firewall_api import Firewall, LIKE, EQ, NOT

# Read configuration
config = configparser.ConfigParser(interpolation=None)
config.read('../Credentials/config.ini')

# Get firewall credentials from config
username = config['Firewall']['username']
password = config['Firewall']['password']
firewall_ip = config['Firewall']['firewall_ip']
port = config['Firewall'].getint('port', fallback=4444)
certificate_verify = config['Firewall'].getboolean('certificate_verify', fallback=False)
timeout = config['Firewall'].getint('timeout', fallback=30)

firewall = Firewall(username, password, firewall_ip, port, certificate_verify, timeout)

In [ ]:
#
# Set IPHost IPs/Networks for Zoom
#

with open("Zoom.txt", "rt") as f:
    ip_addresses_list = f.read().splitlines()

preffix = "Zoom_"
ip_host_group_name = preffix + "IPHost_Group"

entity_type = "IPHostGroup"
entity_data = {
    "Name": ip_host_group_name,
    "IPFamily": "IPv4",
}

print(entity_data)

# print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")

# result = firewall.create("IPHostGroup", {"Name": "Zoom_IPHostGroup", "IPFamily": "IPv4"})
# print("CREATE :: ", result, "\n")


for item in ip_addresses_list:
    entity_data = {}
    entity_type = "IPHost"
    entity_data["IPFamily"] = "IPv4"  # IPv4 is Default settings

    network = ipaddress.IPv4Network(item)

    if str(network.netmask) == "255.255.255.255":
        entity_data["Name"] = preffix + "IP_" + item
        entity_data["HostType"] = "IP"
        entity_data["IPAddress"] = str(network.network_address)
    else:
        entity_data["Name"] = preffix + "NW_" + item
        entity_data["HostType"] = "Network"
        entity_data["IPAddress"] = str(network.network_address)
        entity_data["Subnet"] = str(network.netmask)

    entity_data["HostGroupList"] = {"HostGroup": ip_host_group_name}

    print(entity_data)
    # print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")  # Create IPHost entity_type using entity_data

firewall.close()

In [ ]:
#
# Set IPHost IPs/Networks for ZoomPhone
#

with open("ZoomPhone.txt", "rt") as f:
    ip_addresses_list = f.read().splitlines()

preffix = "ZoomPhone_"
ip_host_group_name = preffix + "IPHost_Group"

entity_type = "IPHostGroup"
entity_data = {
    "Name": ip_host_group_name,
    "IPFamily": "IPv4",
}

print(entity_data)
print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")


for item in ip_addresses_list:
    entity_data = {}
    entity_type = "IPHost"
    entity_data["IPFamily"] = "IPv4"  # IPv4 is Default settings

    network = ipaddress.IPv4Network(item)

    if str(network.netmask) == "255.255.255.255":
        entity_data["Name"] = preffix + "IP_" + item
        entity_data["HostType"] = "IP"
        entity_data["IPAddress"] = str(network.network_address)
    else:
        entity_data["Name"] = preffix + "NW_" + item
        entity_data["HostType"] = "Network"
        entity_data["IPAddress"] = str(network.network_address)
        entity_data["Subnet"] = str(network.netmask)

    entity_data["HostGroupList"] = {"HostGroup": ip_host_group_name}

    print(entity_data)
    print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")  # Create IPHost entity_type using entity_data

firewall.close()

In [ ]:
#
#   PRINT
#


entity_name = "Zoom"
entity_type = "IPHostGroup"
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)

print()

entity_type = "IPHost"
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)

firewall.close()

In [ ]:
#
#   DELETE
#


entity_name = "Zoom"
entity_type = "IPHostGroup"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))

entity_type = "IPHost"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))

firewall.close()